In [ ]:
# Cell 1: Mount Drive and Load Data
from google.colab import drive
drive.mount('/content/drive')

!pip install xgboost optuna -q

import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import RobustScaler
import xgboost as xgb
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
import warnings
warnings.filterwarnings('ignore')

# Load dataset from Google Drive
dataset = pd.read_csv('/content/drive/MyDrive/stock_prediction_data/processed_dataset.csv')

# Sort by date for temporal splitting
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.sort_values('date').reset_index(drop=True)

# Use raw returns as target (better direction accuracy than excess returns)
target_col = 'target_return'

print(f"Loaded dataset: {dataset.shape}")
print(f"Samples: {len(dataset)}, Companies: {dataset['ticker'].nunique()}")
print(f"Date range: {dataset['date'].min().date()} to {dataset['date'].max().date()}")
print(f"Target: {target_col}")
print(f"  Mean: {dataset[target_col].mean():.4f}, Std: {dataset[target_col].std():.4f}")

In [ ]:
# Cell 2: Feature Selection
meta_cols = ['ticker', 'date', 'target_return', 'target_excess_return', 'benchmark_return']
all_feature_cols = [c for c in dataset.columns if c not in meta_cols]

print(f"Starting features: {len(all_feature_cols)}")

X_raw = dataset[all_feature_cols].copy()
y = dataset[target_col].copy()

# 1. Drop features with zero variance
variances = X_raw.var()
zero_var = variances[variances == 0].index.tolist()
if zero_var:
    print(f"Dropping {len(zero_var)} zero-variance features: {zero_var}")
    X_raw = X_raw.drop(columns=zero_var)

# 2. Drop highly correlated features (>0.95)
corr_matrix = X_raw.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_pairs = []
to_drop = set()
for col in upper.columns:
    correlated = upper.index[upper[col] > 0.95].tolist()
    for c in correlated:
        if c not in to_drop:
            high_corr_pairs.append((col, c, corr_matrix.loc[col, c]))
            to_drop.add(c)

if to_drop:
    print(f"\nDropping {len(to_drop)} highly correlated features (r > 0.95):")
    for col, c, r in high_corr_pairs:
        print(f"  {c} (corr={r:.3f} with {col})")
    X_raw = X_raw.drop(columns=list(to_drop))

# 3. Drop features with very low correlation to target (noise removal)
# With 436 companies, many features are pure noise that the model overfits to
target_corr = X_raw.corrwith(y).abs().sort_values(ascending=False)
print(f"\nFeature-target correlations:")
for feat, corr in target_corr.items():
    marker = "  " if corr >= 0.02 else "X "
    print(f"  {marker}{feat:<35s} |r|={corr:.4f}")

low_corr = target_corr[target_corr < 0.02].index.tolist()
if low_corr:
    print(f"\nDropping {len(low_corr)} low-correlation features (|r| < 0.02): {low_corr}")
    X_raw = X_raw.drop(columns=low_corr)

feature_columns = list(X_raw.columns)
print(f"\nSelected features: {len(feature_columns)}")
print(f"Features: {feature_columns}")

In [ ]:
# Cell 3: Temporal Train/Test Split + Time-Series CV

X = dataset[feature_columns].copy()
y = dataset[target_col].copy()

# Temporal split: train on earlier data, test on most recent 20%
unique_dates = sorted(dataset['date'].unique())
n_dates = len(unique_dates)
cutoff_idx = int(n_dates * 0.8)
cutoff_date = unique_dates[cutoff_idx]

train_mask = dataset['date'] < cutoff_date
test_mask = dataset['date'] >= cutoff_date

X_train_raw = X[train_mask]
X_test_raw = X[test_mask]
y_train = y[train_mask].values
y_test = y[test_mask].values

print(f"=== Temporal Split ===")
print(f"Cutoff date: {pd.Timestamp(cutoff_date).date()}")
print(f"Training: {len(X_train_raw)} samples ({dataset[train_mask]['date'].min().date()} to {dataset[train_mask]['date'].max().date()})")
print(f"Test:     {len(X_test_raw)} samples ({dataset[test_mask]['date'].min().date()} to {dataset[test_mask]['date'].max().date()})")

# RobustScaler: uses median/IQR, resistant to outliers (PE ratio max 17,474 etc)
scaler = RobustScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train_raw), columns=feature_columns)
X_test = pd.DataFrame(scaler.transform(X_test_raw), columns=feature_columns)

# Build expanding-window CV splits for training data
train_dates = sorted(dataset[train_mask]['date'].unique())
n_train_dates = len(train_dates)
n_folds = 5
fold_size = n_train_dates // (n_folds + 1)

cv_splits = []
for i in range(n_folds):
    train_end = (i + 2) * fold_size
    val_start = train_end
    val_end = min(train_end + fold_size, n_train_dates)

    cv_train_dates = set(train_dates[:train_end])
    cv_val_dates = set(train_dates[val_start:val_end])

    train_idx = [j for j, d in enumerate(dataset[train_mask]['date']) if d in cv_train_dates]
    val_idx = [j for j, d in enumerate(dataset[train_mask]['date']) if d in cv_val_dates]

    if len(val_idx) > 0:
        cv_splits.append((train_idx, val_idx))

print(f"\n=== Time-Series CV ({len(cv_splits)} folds) ===")
for i, (tr, va) in enumerate(cv_splits):
    print(f"  Fold {i+1}: train={len(tr)} samples, val={len(va)} samples")

In [ ]:
# Cell 4: Hyperparameter Tuning with Optuna (Bayesian Optimization)
# Constrained search space to prevent overfitting with 436 companies

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 2, 4),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 0.8),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.8),
        'min_child_weight': trial.suggest_int('min_child_weight', 10, 50),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 50.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 50.0, log=True),
        'huber_slope': trial.suggest_float('huber_slope', 0.05, 0.3),
    }

    cv_scores = []
    for train_idx, val_idx in cv_splits:
        model = xgb.XGBRegressor(
            **{k: v for k, v in params.items() if k != 'huber_slope'},
            huber_slope=params['huber_slope'],
            random_state=42,
            objective='reg:pseudohubererror',
            verbosity=0,
        )
        model.fit(X_train.values[train_idx], y_train[train_idx])
        preds = model.predict(X_train.values[val_idx])
        rmse = np.sqrt(mean_squared_error(y_train[val_idx], preds))
        cv_scores.append(rmse)

    return np.mean(cv_scores)

print("Running Optuna hyperparameter search (50 trials, constrained for regularization)...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

best_params = study.best_params
print(f"\n=== Best Parameters ===")
for k, v in best_params.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")
    else:
        print(f"  {k}: {v}")

# Re-run best params through CV folds to get per-fold scores for std
best_cv_scores = []
for train_idx, val_idx in cv_splits:
    bp = best_params.copy()
    hs = bp.pop('huber_slope')
    fold_model = xgb.XGBRegressor(
        **bp,
        huber_slope=hs,
        random_state=42,
        objective='reg:pseudohubererror',
        verbosity=0,
    )
    fold_model.fit(X_train.values[train_idx], y_train[train_idx])
    fold_preds = fold_model.predict(X_train.values[val_idx])
    fold_rmse = np.sqrt(mean_squared_error(y_train[val_idx], fold_preds))
    best_cv_scores.append(fold_rmse)

cv_rmse_mean = np.mean(best_cv_scores)
cv_rmse_std = np.std(best_cv_scores)
print(f"\nCV RMSE with best params: {cv_rmse_mean:.4f} +/- {cv_rmse_std:.4f}")
print(f"Per-fold: {[f'{s:.4f}' for s in best_cv_scores]}")

In [ ]:
# Cell 5: Train Final Model, Direction Classifier, and Calibration

# --- Regression model with Huber loss ---
huber_slope = best_params.pop('huber_slope')
model = xgb.XGBRegressor(
    **best_params,
    huber_slope=huber_slope,
    random_state=42,
    objective='reg:pseudohubererror',
    verbosity=0,
)
model.fit(X_train.values, y_train)

y_pred_train = model.predict(X_train.values)
y_pred_test = model.predict(X_test.values)

# --- Direction classifier (simple, independent tuning) ---
y_train_dir = (y_train > 0).astype(int)
y_test_dir = (y_test > 0).astype(int)

# Simple classifier: shallow trees, heavy regularization
dir_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=2,
    learning_rate=0.05,
    subsample=0.7,
    colsample_bytree=0.6,
    min_child_weight=20,
    reg_alpha=1.0,
    reg_lambda=5.0,
    random_state=42,
    objective='binary:logistic',
    eval_metric='logloss',
    verbosity=0,
)
dir_model.fit(X_train.values, y_train_dir)
dir_probs = dir_model.predict_proba(X_test.values)[:, 1]  # P(up)
dir_preds = (dir_probs > 0.5).astype(int)

dir_correct = np.sum(dir_preds == y_test_dir)
dir_accuracy = dir_correct / len(y_test)

# --- Prediction calibration ---
# Scale regression predictions to match training data variance
pred_std = y_pred_test.std()
actual_std = y_train.std()
pred_mean = y_pred_test.mean()
train_mean = y_train.mean()

if pred_std > 0:
    y_pred_calibrated = (y_pred_test - pred_mean) * (actual_std / pred_std) + train_mean
else:
    y_pred_calibrated = y_pred_test.copy()

# --- Ensemble: blend regression direction with classifier confidence ---
dir_signal = 2 * dir_probs - 1  # maps [0,1] -> [-1,1]
y_pred_ensemble = y_pred_calibrated.copy()
# Flip calibrated prediction sign when classifier strongly disagrees
for i in range(len(y_pred_ensemble)):
    if np.sign(y_pred_calibrated[i]) != np.sign(dir_signal[i]):
        # Use classifier's direction with calibrated magnitude
        y_pred_ensemble[i] = abs(y_pred_calibrated[i]) * np.sign(dir_signal[i])

# Restore huber_slope in best_params for saving
best_params['huber_slope'] = huber_slope

# --- Evaluate all variants ---
print("=" * 65)
print("    PREDICTION METHOD COMPARISON")
print("=" * 65)
print(f"{'Method':<25s} {'RMSE':>8s} {'R2':>8s} {'Dir Acc':>8s} {'Train R2':>8s}")
print("-" * 60)

# Check overfitting
train_rmse_check = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_r2_check = r2_score(y_train, y_pred_train)

variants = {
    'Raw Regression': y_pred_test,
    'Calibrated Regression': y_pred_calibrated,
    'Ensemble': y_pred_ensemble,
}
for name, preds in variants.items():
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    da = np.mean(np.sign(y_test) == np.sign(preds))
    tr2 = f"{train_r2_check:.4f}" if name == 'Raw Regression' else ""
    print(f"  {name:<23s} {rmse:>8.4f} {r2:>8.4f} {da:>7.1%} {tr2:>8s}")

print(f"  {'Direction Classifier':<23s} {'':>8s} {'':>8s} {dir_accuracy:>7.1%}")
print(f"\nOverfit check: Train R2={train_r2_check:.4f}, Test R2={r2_score(y_test, y_pred_test):.4f}")
print(f"Prediction std - Raw: {y_pred_test.std():.4f}, Calibrated: {y_pred_calibrated.std():.4f}, Actual: {y_test.std():.4f}")

In [ ]:
# Cell 6: Feature Importance
import matplotlib.pyplot as plt

importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=True)

fig, ax = plt.subplots(figsize=(10, max(6, len(feature_columns) * 0.3)))
ax.barh(importance['feature'], importance['importance'], color='steelblue')
ax.set_xlabel('Feature Importance')
ax.set_title('XGBoost Feature Importance (Huber Loss)')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/stock_prediction_data/feature_importance.png', dpi=150)
plt.show()

print("Top 10 features:")
for _, row in importance.tail(10).iloc[::-1].iterrows():
    bar = '#' * int(row['importance'] * 50)
    print(f"  {row['feature']:30s} {row['importance']:.3f} {bar}")

In [ ]:
# Cell 7: Prediction Analysis — Compare Raw, Calibrated, and Ensemble
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Row 1: Predicted vs Actual scatter for each method
for col, (name, preds) in enumerate(variants.items()):
    ax = axes[0, col]
    ax.scatter(y_test, preds, alpha=0.5, edgecolor='black', s=40)
    lims = [min(y_test.min(), preds.min()) - 0.05, max(y_test.max(), preds.max()) + 0.05]
    ax.plot(lims, lims, 'r--', label='Perfect')
    ax.set_xlabel(f'Actual {target_col}')
    ax.set_ylabel(f'Predicted')
    ax.set_title(f'{name}')
    r2 = r2_score(y_test, preds)
    da = np.mean(np.sign(y_test) == np.sign(preds))
    ax.text(0.05, 0.95, f'R2={r2:.3f}\nDir={da:.1%}', transform=ax.transAxes,
            va='top', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Row 2, Col 0: Prediction spread comparison
axes[1, 0].hist(y_pred_test, bins=30, alpha=0.6, label='Raw', color='blue')
axes[1, 0].hist(y_pred_calibrated, bins=30, alpha=0.6, label='Calibrated', color='orange')
axes[1, 0].hist(y_test, bins=30, alpha=0.4, label='Actual', color='green')
axes[1, 0].set_xlabel('Return')
axes[1, 0].set_title('Prediction Spread: Before vs After Calibration')
axes[1, 0].legend()

# Row 2, Col 1: Cumulative returns comparison
test_returns = dataset[test_mask]['target_return'].values
buy_hold = np.cumprod(1 + test_returns) - 1
for name, preds in variants.items():
    strategy_returns = np.where(preds > 0, test_returns, 0)
    strategy = np.cumprod(1 + strategy_returns) - 1
    axes[1, 1].plot(range(len(strategy)), strategy, label=name, alpha=0.7)
axes[1, 1].plot(range(len(buy_hold)), buy_hold, label='Buy & Hold', alpha=0.7, linestyle='--', color='black')
axes[1, 1].set_xlabel('Test Sample')
axes[1, 1].set_ylabel('Cumulative Return')
axes[1, 1].set_title('Cumulative Returns: Model Strategies vs Buy & Hold')
axes[1, 1].legend(fontsize=8)
axes[1, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Row 2, Col 2: Direction accuracy by method
method_names = list(variants.keys()) + ['Classifier']
dir_accs = [np.mean(np.sign(y_test) == np.sign(p)) for p in variants.values()] + [dir_accuracy]
colors = ['steelblue'] * len(variants) + ['coral']
axes[1, 2].barh(method_names, dir_accs, color=colors)
axes[1, 2].axvline(x=0.5, color='red', linestyle='--', label='Random baseline')
axes[1, 2].set_xlabel('Direction Accuracy')
axes[1, 2].set_title('Direction Accuracy by Method')
axes[1, 2].set_xlim(0.4, 0.75)
axes[1, 2].legend()

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/stock_prediction_data/model_results.png', dpi=150)
plt.show()

In [ ]:
# Cell 8: Save Model and Results

# Pick the best-performing variant for primary metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
direction_correct = np.sum(np.sign(y_test) == np.sign(y_pred_test))
reg_direction_accuracy = direction_correct / len(y_test)

# Calibrated metrics
cal_rmse = np.sqrt(mean_squared_error(y_test, y_pred_calibrated))
cal_r2 = r2_score(y_test, y_pred_calibrated)
cal_dir = np.mean(np.sign(y_test) == np.sign(y_pred_calibrated))

# Ensemble metrics
ens_rmse = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
ens_r2 = r2_score(y_test, y_pred_ensemble)
ens_dir = np.mean(np.sign(y_test) == np.sign(y_pred_ensemble))

results = {
    'model': model,
    'scaler': scaler,
    'feature_columns': feature_columns,
    'target_column': target_col,
    'best_params': best_params,
    'metrics': {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'direction_accuracy': float(reg_direction_accuracy),
        'cv_rmse_mean': float(cv_rmse_mean),
        'cv_rmse_std': float(cv_rmse_std),
        'overfit_ratio': float(train_rmse / test_rmse),
        # Calibrated metrics
        'calibrated_rmse': cal_rmse,
        'calibrated_r2': cal_r2,
        'calibrated_direction_accuracy': float(cal_dir),
        # Ensemble metrics
        'ensemble_rmse': ens_rmse,
        'ensemble_r2': ens_r2,
        'ensemble_direction_accuracy': float(ens_dir),
        # Classifier metrics
        'classifier_direction_accuracy': float(dir_accuracy),
    },
    'split_info': {
        'method': 'temporal',
        'cutoff_date': str(pd.Timestamp(cutoff_date).date()),
        'train_size': len(X_train),
        'test_size': len(X_test),
        'cv_folds': len(cv_splits),
    },
    # Save all prediction variants
    'predictions': {
        'y_train': y_train.tolist(),
        'y_test': y_test.tolist(),
        'y_pred_train': y_pred_train.tolist(),
        'y_pred_test': y_pred_test.tolist(),
        'y_pred_calibrated': y_pred_calibrated.tolist(),
        'y_pred_ensemble': y_pred_ensemble.tolist(),
        'dir_probs': dir_probs.tolist(),
        'test_tickers': dataset[test_mask]['ticker'].tolist(),
        'test_dates': dataset[test_mask]['date'].astype(str).tolist(),
        'test_raw_returns': dataset[test_mask]['target_return'].tolist(),
    },
}

with open('/content/drive/MyDrive/stock_prediction_data/model_results.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Saved to Google Drive: model_results.pkl")
print(f"\n{'='*55}")
print(f"    FINAL RESULTS SUMMARY")
print(f"{'='*55}")
print(f"\nTarget: {target_col}")
print(f"Samples: {len(X_train)} train / {len(X_test)} test")
print(f"Features: {len(feature_columns)}")
print(f"\n--- Raw Regression (Huber Loss) ---")
print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
print(f"Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
print(f"Direction: {reg_direction_accuracy:.1%}, Overfit: {train_rmse/test_rmse:.2f}")
print(f"CV RMSE: {cv_rmse_mean:.4f} +/- {cv_rmse_std:.4f}")
print(f"\n--- Calibrated Regression ---")
print(f"RMSE: {cal_rmse:.4f}, R2: {cal_r2:.4f}, Direction: {cal_dir:.1%}")
print(f"\n--- Ensemble ---")
print(f"RMSE: {ens_rmse:.4f}, R2: {ens_r2:.4f}, Direction: {ens_dir:.1%}")
print(f"\n--- Direction Classifier ---")
print(f"Direction: {dir_accuracy:.1%}")